<a href="https://colab.research.google.com/github/jjablonski-it/pjatk-mhe/blob/main/MHE_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import random
import matplotlib.pyplot as plt
from time import time

In [50]:
# constants
length = 1000
value_range = 100_000_000

In [51]:
data = sorted([random.randint(-value_range,value_range) for _ in range(length)])
# save data to file data.txt
with open('data.txt','w') as f:
    for i in data:
        f.write(str(i)+'\n')

In [52]:
# functions
def random_index():
  return ''.join(['%d' % random.randint(0,1) for _ in range(length)])

def index_to_set(index):
  result = []
  for i, x in enumerate(list(index)):
    if(x=='1'):
      result.append(i)
  return list(map(lambda x: data[x] ,result))

def subset_sum(index):
  return abs(sum(index_to_set(index)))

def negate_bit(index, n):
  index_list = list(index)
  bit = index_list[n]
  index_list[n] = '1' if bit=='0' else '0'
  return ''.join(index_list)

def generate_neighbours(index):
  return filter(lambda x: x != '0'*length, [negate_bit(index, x) for x in range(length)])

# def get_best_index(indexes):
#   sum_dict = {index: subset_sum(index) for index in indexes}
#   return min(sum_dict, key=sum_dict.get)

In [53]:
def tabu_search(index=random_index(), iterations_limit=100): # with going back
  tl = set() # tabu list
  l  = [index] # steps list
  best_index = index
  current_index = index
  i = 0
  while(i < iterations_limit and subset_sum(best_index)!=0 and l):
    i += 1
    # print(i, 'index', current_index, 'sum', subset_sum(current_index))
    neighbours = generate_neighbours(current_index)
    neighbour_sum_dict = {index: subset_sum(index) for index in neighbours} # {index -> sum} dict
    sorted_neighbours = map(lambda x: x[0], sorted(neighbour_sum_dict.items(), key=lambda item: item[1])) # [index] array
    valid_neighbours = list(filter(lambda x: x not in tl, sorted_neighbours))

    tl.add(current_index)
    if(valid_neighbours):
      best_local_index = valid_neighbours.pop(0)
      if(subset_sum(best_local_index) < subset_sum(best_index)):
        l.append(current_index)
        best_index = best_local_index
      current_index = best_local_index
    else:
      current_index = l.pop()
  print('iterations', i)
  return best_index

In [54]:
start_index = random_index()
# save start index to file start_index.txt
with open('start_index.txt','w') as f:
    f.write(start_index)
print('start_index', start_index)
print('tabu_search')
start_time = time()
best_index = tabu_search(start_index, 100000)
duration = time() - start_time
print('start_index sum %d' % subset_sum(start_index))
print('best_index  sum %d' % subset_sum(best_index))
print('duration', duration)


start_index 0100110111100110100100111010000010010100111111110011111011011100011111100110000011101001110011101001100111000001011100101000111110111010100010010101100000011111110100011010011101110101011111001011011010011110110111101111001001111101100100000100100011001000011100111001100011001001001010101100100111001010000110001011011110110100110011001001010001010011001000110111000011011101100010111101000110101100011111011010100010110111000111001001111011011011111100111100111000001111001011111100011000011010101001100100001000111011000110000000101111011110010011101110111110100110001001001111010100101111010001100010110000110001001110110101110111010000111011001010010001110000001110111101011001010011010001010011111000110010010010010000000110111000111000010110010001100011100110100110011001000101000111010101110011101101101101000100100101101100011101100110000011010111110111001101100001110111100101100010100111001100100001010100011110000100110110100100010111001000000100110011000000000100011011110010

| Details | Py | Cpp |
| - | - | - |
|Length: 1000
Range: 100 mln
Iterations: 1000| 85.642| 39.906 |
| - | - | - |
|Length: 1000
Range: 100 mln
Iterations: 100000| 5968.423 | 12481.957 |

Cpp ok 2x szybciej